# Analyse Exploratoire

### Import des modules

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np

ModuleNotFoundError: No module named 'pandas'

### Analyse Exploratoire

In [ ]:
building_consumption = pd.read_csv("building_energy_benchmarking.csv")

In [ ]:
# On regarde comment un batiment est défini dans ce jeu de données 
building_consumption.head()

In [ ]:
# On regarde le nombre de valeurs manquantes par colonne ainsi que leur type 
building_consumption.info()

In [ ]:
building_consumption["DataYear"].nunique()


In [ ]:
building_consumption["City"].nunique()

In [ ]:
building_consumption["State"].nunique()

In [ ]:
building_consumption["Comments"].nunique()

In [ ]:
building_consumption["CouncilDistrictCode"].nunique()

In [ ]:
building_consumption["ZipCode"].nunique()

On peut supprimer les colonnes **DataYear**, **City**, **State** car elles ont toutes la même valeur. et **Comments** car il n'a pas de valeur.

In [ ]:
building_consumption["Neighborhood"].value_counts()

Renommons les colonnes qui correspondent au même quartier

In [ ]:
building_consumption["BuildingType"].value_counts()

On ne garde que les lignes concernant les bâtiments Non Résidentiels, sauf **Nonresidantial WA** qui correspont à Washington

In [ ]:
building_consumption_clean = building_consumption.copy()

In [ ]:
building_consumption_clean["Neighborhood"] = (
    building_consumption_clean["Neighborhood"].replace({
        'Central': 'CENTRAL',
        'North': 'NORTH',
        'Northwest': 'NORTHWEST',
        'Ballard': 'BALLARD',
        'Delridge': 'DELRIDGE',
        'DELRIDGE NEIGHBOHORDOODS': 'DELRIDGE'
    })
)


In [ ]:
building_consumption_clean = building_consumption_clean[
    building_consumption_clean["BuildingType"].isin(["NonResidential", "Nonresidential COS"])
]

In [ ]:
building_consumption_clean = building_consumption_clean.drop(["DataYear", "City", "State", "Comments"], axis=1)


In [ ]:
building_consumption_clean = building_consumption_clean.drop(["Latitude","Longitude"], axis=1)

Les données d'énergies principales sont en kBtu, on peut donc supprimer les colonnes en kWh et en therms

In [ ]:
building_consumption_clean = building_consumption_clean.drop(["Electricity(kWh)", "NaturalGas(therms)"], axis=1)

In [8]:
building_consumption_clean.columns

NameError: name 'building_consumption_clean' is not defined

DefaultData : Données estimées ou réelles

In [ ]:
building_consumption_clean["YearsENERGYSTARCertified"].value_counts()

In [ ]:
building_consumption_clean["YearsENERGYSTARCertified"] = (
    building_consumption_clean["YearsENERGYSTARCertified"].astype(str).str.extract(r"(\d{4})")
)

In [ ]:
building_consumption_clean["DefaultData"].value_counts()

ComplianceStatus : Conformité réglementaire du bâtiment

In [ ]:
building_consumption_clean["ComplianceStatus"].value_counts()


In [ ]:
building_consumption_clean[building_consumption_clean["ComplianceStatus"] == "Error - Correct Default Data"]

Ces bâtiment ont beaucoup de données. On va donc les garder.

In [ ]:
building_consumption_clean[building_consumption_clean["ComplianceStatus"] == "Missing Data"]

On peut supprimer ces lignes qui ont trop de valeurs manquantes.

In [ ]:
building_consumption_clean = building_consumption_clean[
    ~building_consumption_clean["ComplianceStatus"].isin(["Missing Data"])
]

Outlier : Valeurs extrêmes identifiées

In [ ]:
building_consumption_clean["Outlier"].value_counts()

Garder les valeurs qui prennent en compte la météo

In [ ]:
building_consumption_clean = building_consumption_clean.drop(["SourceEUIWN(kBtu/sf)", "SourceEUI(kBtu/sf)"], axis=1)

In [ ]:
building_consumption_clean["SiteEUIWN(kBtu/sf)"].min()

In [ ]:
building_consumption_clean.loc[building_consumption_clean["SiteEUIWN(kBtu/sf)"].idxmin()]


Ce bâtiment n'a pas de données concernant SourceEUIWN(kBtu/sf) , mais a des données en terme de SteamUse(kBtu) et Electricity(kBtu) , on va donc le garder.

In [ ]:
building_consumption_clean["SiteEUIWN(kBtu/sf)"].mean()

In [ ]:
building_consumption_clean["SiteEUIWN(kBtu/sf)"].max()

In [ ]:
building_consumption_clean.loc[building_consumption_clean["SiteEUIWN(kBtu/sf)"].idxmax()]

Ok pour le max, c'est un DataCenter

In [9]:
building_consumption_clean.info()

NameError: name 'building_consumption_clean' is not defined

In [ ]:
building_consumption_clean.shape

In [ ]:
building_consumption_clean["LargestPropertyUseType"].unique()

In [ ]:
cols_to_check = [
    "SiteEUIWN(kBtu/sf)",
    "SiteEnergyUse(kBtu)",
    "SiteEnergyUseWN(kBtu)",
    "SteamUse(kBtu)",
    "Electricity(kBtu)",
    "NaturalGas(kBtu)"
]

In [ ]:
building_consumption_clean[
    building_consumption_clean[cols_to_check].isna().all(axis=1) |  
    (building_consumption_clean[cols_to_check] == 0).all(axis=1) 
]

On peut supprimer ces lignes qui n'ont aucunes données.

In [ ]:
building_consumption_clean = building_consumption_clean.drop([304, 578, 2670])


In [ ]:
building_consumption_clean[["SiteEnergyUse(kBtu)", "SiteEnergyUseWN(kBtu)"]].describe()

In [ ]:
df = building_consumption_clean

plt.figure(figsize=(10,6))
sns.regplot(
    x=df["SiteEnergyUse(kBtu)"],
    y=df["SiteEnergyUseWN(kBtu)"]
)

plt.xlabel("SiteEnergyUse(kBtu)")
plt.ylabel("SiteEnergyUseWN(kBtu)")
plt.title("Relation entre SiteEnergyUse(kBtu) et  SiteEnergyUseWN(kBtu)")

plt.tight_layout()
plt.show()


Il semble qu'il  y ait des données énergétique égale à 0. 

In [ ]:
mask = (
    (building_consumption_clean["SiteEnergyUse(kBtu)"].isna()) |
    (building_consumption_clean["SiteEnergyUseWN(kBtu)"].isna()) |
    (building_consumption_clean["SiteEnergyUse(kBtu)"] == 0) |
    (building_consumption_clean["SiteEnergyUseWN(kBtu)"] == 0)
)

building_consumption_clean[mask]

In [ ]:
building_consumption_clean[["SiteEnergyUse(kBtu)", "SiteEnergyUseWN(kBtu)"]].describe()

Aucune ligne n'est égale à 0, c'est le rapport d'échelle qui donne cette impression

Calculons le ratio entre **SiteEnergyUse(kBtu)** et **SiteEnergyUseWN(kBtu)** pour remplir les lignes sans données.

In [ ]:
clean_df = df[["SiteEnergyUse(kBtu)", "SiteEnergyUseWN(kBtu)"]].copy()

clean_df["SiteEnergyUseWN(kBtu)"].replace(0, np.nan, inplace=True)

clean_df["ratio"] = clean_df["SiteEnergyUse(kBtu)"] / clean_df["SiteEnergyUseWN(kBtu)"]

clean_df = clean_df.dropna(subset=["ratio"])

clean_df["ratio"].mean()

Le ratio est de 0.95 entre SiteEUIWN(kBtu/sf) & SiteEUI(kBtu/sf) est de 0.95 , on peut donc remplir les valeurs manquantes.

In [ ]:
ratio_mean = 0.95

building_consumption_clean.loc[
    (building_consumption_clean["SiteEnergyUse(kBtu)"].isna() | (building_consumption_clean["SiteEnergyUse(kBtu)"] == 0)) &
    (building_consumption_clean["SiteEnergyUseWN(kBtu)"].notna() & (building_consumption_clean["SiteEnergyUseWN(kBtu)"] != 0)),
    "SiteEnergyUse(kBtu)"
] = building_consumption_clean["SiteEnergyUseWN(kBtu)"] * ratio_mean


In [ ]:
building_consumption_clean.loc[
    (building_consumption_clean["SiteEnergyUseWN(kBtu)"].isna() | (building_consumption_clean["SiteEnergyUseWN(kBtu)"] == 0)) &
    (building_consumption_clean["SiteEnergyUse(kBtu)"].notna() & (building_consumption_clean["SiteEnergyUse(kBtu)"] != 0)),
    "SiteEnergyUseWN(kBtu)"
] = building_consumption_clean["SiteEUI(kBtu/sf)"] / ratio_mean

Lien entre l'année de construction et la consommation énergétique

In [ ]:
df = building_consumption_clean

plt.figure(figsize=(10,6))
sns.regplot(
    x=df["YearBuilt"],
    y=df["SiteEnergyUseWN(kBtu)"]
)

plt.xlabel("YearBuilt")
plt.ylabel("SiteEnergyUseWN(kBtu)")
plt.title("Relation entre YearBuilt et  SiteEnergyUseWN(kBtu)")

plt.tight_layout()
plt.show()

In [ ]:
clean_df = df[["SiteEnergyUseWN(kBtu)", "YearBuilt"]].copy()

clean_df["SiteEnergyUseWN(kBtu)"].replace(0, np.nan, inplace=True)

clean_df["ratio"] =   clean_df["YearBuilt"] / clean_df["SiteEnergyUseWN(kBtu)"]

clean_df = clean_df.dropna(subset=["ratio"])

clean_df["ratio"].mean()

Il y a un ratio de 0.22 entre l'année de construction et sa consommation énergétique

In [ ]:
df = building_consumption_clean

plt.figure(figsize=(10,6))
sns.regplot(
    x=df["PropertyGFATotal"],
    y=df["SiteEnergyUseWN(kBtu)"]
)

plt.xlabel("PropertyGFATotal")
plt.ylabel("SiteEnergyUseWN(kBtu)")
plt.title("Relation entre PropertyGFATotal et  SiteEnergyUseWN(kBtu)")

plt.tight_layout()
plt.show()

On ne peut pas faire de lien entre la surface totale & la consommation énergétique

In [ ]:
mask = (
    (building_consumption_clean["TotalGHGEmissions"].isna()) |
    (building_consumption_clean["TotalGHGEmissions"] == 0) 
)

building_consumption_clean[mask]

On peut supprimer cette ligne car nous n'avons pas d'info sur ses émission de CO2

In [10]:
building_consumption_clean = building_consumption_clean.drop([513])


NameError: name 'building_consumption_clean' is not defined

In [ ]:
df = building_consumption_clean

plt.figure(figsize=(10,6))
sns.regplot(
    x=df["TotalGHGEmissions"],
    y=df["SiteEnergyUseWN(kBtu)"]
)

plt.xlabel("TotalGHGEmissionsl")
plt.ylabel("SiteEnergyUseWN(kBtu)")
plt.title("Relation entre TotalGHGEmissions et  SiteEnergyUseWN(kBtu)")

plt.tight_layout()
plt.show()


In [ ]:
df = building_consumption_clean

plt.figure(figsize=(10,6))
sns.regplot(
    x=df["LargestPropertyUseTypeGFA"],
    y=df["SiteEnergyUseWN(kBtu)"]
)

plt.xlabel("LargestPropertyUseTypeGFA")
plt.ylabel("SiteEnergyUseWN(kBtu)")
plt.title("Relation entre LargestPropertyUseTypeGFA et  SiteEnergyUseWN(kBtu)")

plt.tight_layout()
plt.show()

Regardons quel type de bâtiment consomment le plus.

In [ ]:
df = building_consumption_clean

seuil = df["SiteEnergyUseWN(kBtu)"].quantile(0.75)

top_25 = df[df["SiteEnergyUseWN(kBtu)"] >= seuil]


top_25["LargestPropertyUseType"].describe()

In [ ]:
counts = top_25["LargestPropertyUseType"].value_counts()

# 4. Barplot
plt.figure(figsize=(12,6))
plt.bar(counts.index, counts.values)

plt.xlabel("LargestPropertyUseType")
plt.ylabel("Count")
plt.title("Répartition des usages (LargestPropertyUseType) dans le Top 25% des bâtiments (SiteEnergyUseWN)")
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

On remarque que ce sont les bureaux qui consomment le plus. Regardons plus en détail ces bureaux.

In [ ]:
df = building_consumption_clean[building_consumption_clean["LargestPropertyUseType"] == "Office"]

seuil = df["SiteEnergyUseWN(kBtu)"].quantile(0.75)

top_25 = df[df["SiteEnergyUseWN(kBtu)"] >= seuil]


top_25["SiteEnergyUseWN(kBtu)"].describe()

In [ ]:
plt.figure(figsize=(8,6))
plt.boxplot(
    [top_25["PropertyGFATotal"]],

    labels=["Top 25% Consommation"] 
)
plt.title("Comparaison PropertyGFATotal selon le niveau de consommation (Office)")
plt.ylabel("Superficie (GFA)")
plt.show()

Regardons plus en détail les bâtiment où la superficie est supérieure à 1

In [ ]:
oversized_building = top_25[top_25["PropertyGFATotal"] > 1] 
oversized_building

In [ ]:
oversized_building["PrimaryPropertyType"].unique()

Regardons plus en détail les **Small- and Mid-Sized Office**

In [ ]:
oversized_building[
    oversized_building["PrimaryPropertyType"] == "Small- and Mid-Sized Office"
]

In [ ]:
office = oversized_building[
    oversized_building["PrimaryPropertyType"] == "Small- and Mid-Sized Office"
].describe()
office

In [ ]:
df_office = oversized_building[
    oversized_building["PrimaryPropertyType"] == "Small- and Mid-Sized Office"
]

plt.figure(figsize=(8,6))

plt.boxplot(
    df_office["LargestPropertyUseTypeGFA"],
    labels=["Small- and Mid-Sized Office"]
)

plt.title("Distribution de LargestPropertyUseTypeGFA pour Small- and Mid-Sized Office")
plt.ylabel("Superficie (GFA)")
plt.show()

In [ ]:
office["SiteEnergyUseWN(kBtu)"].describe()


In [ ]:
plt.boxplot(
    office["SiteEnergyUseWN(kBtu)"],
    labels=["Small- and Mid-Sized Office"]
)
plt.title("Distribution de SiteEnergyUseWN(kBtu) pour Small- and Mid-Sized Office")
plt.ylabel("Consommation d'énergie (kBtu)")
plt.show()

In [ ]:
index_max_energie = df_office["SiteEnergyUse(kBtu)"].idxmax()
building_consumption_clean.loc[index_max_energie]

In [ ]:
building_consumption_clean.loc[index_max_energie, "SiteEnergyUseWN(kBtu)"] / building_consumption_clean.loc[index_max_energie, "LargestPropertyUseTypeGFA"]

Le ratio est de 467. Ce bâtiment consomme énormément par rapport à sa surface. Mais on peut remarquer qu'il a été construit en 1929.

In [ ]:
colonnes_correlation = [
    'PropertyGFATotal', 
    'YearBuilt', 
    'NumberofFloors', 
    'ENERGYSTARScore', 
    'SiteEUI(kBtu/sf)', 
    'SiteEnergyUse(kBtu)', 
    'TotalGHGEmissions',
]

df_corr = building_consumption_clean[colonnes_correlation].copy()

matrice_corr = df_corr.corr(method='pearson')

plt.figure(figsize=(10, 8))
sns.heatmap(
    matrice_corr, 
    annot=True, 
    cmap='coolwarm', 
    fmt=".2f",
    linewidths=.5,
)
plt.title('Matrice de Corrélation')
plt.show()


In [11]:
building_consumption_clean.boxplot(
    column="SiteEUIWN(kBtu/sf)",
    by="Neighborhood",
    figsize=(15,6),
    rot=90
)
plt.title("Distribution de SiteEUIWN(kBtu/sf)  par Neighborhood")
plt.suptitle("") 
plt.ylabel("SiteEUIWN(kBtu/sf)")
plt.xlabel("Neighborhood")
plt.show()

NameError: name 'building_consumption_clean' is not defined

In [ ]:
building_consumption_clean.boxplot(
    column="YearBuilt",
    by="Neighborhood",
    figsize=(15,6),
    rot=90
)
plt.title("Distribution de YearBuilt par Neighborhood")
plt.suptitle("")  # Supprime le titre par défaut
plt.ylabel("YearBuilt")
plt.xlabel("Neighborhood")
plt.show()

In [ ]:
df = building_consumption_clean

plt.figure(figsize=(10,6))
sns.regplot(
    x=df["SiteEnergyUse(kBtu)"],
    y=df["TotalGHGEmissions"]
)

plt.xlabel("SiteEnergyUse(kBtu)")
plt.ylabel("SiteEnergyUseWN(kBtu)")
plt.title("Relation entre SiteEnergyUse(kBtu) et TotalGHGEmissions")

plt.tight_layout()
plt.show()

In [ ]:
building_consumption_clean.boxplot(
    column="SiteEUIWN(kBtu/sf)",
    by="LargestPropertyUseType",
    figsize=(15,6),
    rot=90
)
plt.title("Distribution de SiteEUIWN(kBtu/sf) par LargestPropertyUseType")
plt.suptitle("")  # Supprime le titre par défaut
plt.ylabel("SiteEUIWN(kBtu/sf)")
plt.xlabel("LargestPropertyUseType")
plt.show()

In [ ]:

top_10_types = building_consumption_clean['PrimaryPropertyType'].value_counts().nlargest(10).index
df_top_types = building_consumption_clean[building_consumption_clean['PrimaryPropertyType'].isin(top_10_types)]


plt.figure(figsize=(14, 8))

sns.boxplot(
    x='PrimaryPropertyType',
    y='SiteEnergyUse(kBtu)',
    data=df_top_types,
    palette='viridis'
)


plt.title("Distribution de la Consommation d'Énergie (SiteEnergyUse) par Type de Propriété (Top 10)")
plt.xlabel("Type de Propriété Principal")
plt.ylabel("Consommation d'Énergie (kBtu)")
plt.xticks(rotation=45, ha='right') 
plt.tight_layout() 
plt.show()

#### TERMINER L'ANALYSE EXPLORATOIRE 

A réaliser : 
- Une analyse descriptive des données, y compris une explication du sens des colonnes gardées, des arguments derrière la suppression de lignes ou de colonnes, des statistiques descriptives et des visualisations pertinentes.

# Modélisation 

### Import des modules 

In [ ]:
#Selection
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV, 
    cross_validate,
)
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error 
from sklearn.inspection import permutation_importance

#Preprocess
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

#Modèles
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor


### Feature Engineering

A réaliser : Enrichir le jeu de données actuel avec de nouvelles features issues de celles existantes. 

In [ ]:
# CODE FEATURE ENGINEERING

### Préparation des features pour la modélisation

A réaliser :
* Si ce n'est pas déjà fait, supprimer toutes les colonnes peu pertinentes pour la modélisation.
* Tracer la distribution de la cible pour vous familiariser avec l'ordre de grandeur. En cas d'outliers, mettez en place une démarche pour les supprimer.
* Débarrassez-vous des features redondantes en utilisant une matrice de corrélation.
* Réalisez différents graphiques pour comprendre le lien entre vos features et la target (boxplots, scatterplots, pairplot si votre nombre de features numériques n'est pas très élevé).
*  Séparez votre jeu de données en un Pandas DataFrame X (ensemble de feautures) et Pandas Series y (votre target).
* Si vous avez des features catégorielles, il faut les encoder pour que votre modèle fonctionne.

In [ ]:
# CODE PREPARATION DES FEATURES

### Comparaison de différents modèles supervisés

A réaliser :
* Pour chaque algorithme que vous allez tester, vous devez :
    * Réaliser au préalable une séparation en jeu d'apprentissage et jeu de test via une validation croisée.
    * Si les features quantitatives que vous souhaitez utiliser ont des ordres de grandeur très différents les uns des autres, et que vous utilisez un algorithme de regression qui est sensible à cette différence, alors il faut réaliser un scaling (normalisation) de la donnée au préalable.
    * Entrainer le modèle sur le jeu de Train
    * Prédire la cible sur la donnée de test (nous appelons cette étape, l'inférence).
    * Calculer les métriques de performance R2, MAE et RMSE sur le jeu de train et de test.
    * Interpréter les résultats pour juger de la fiabilité de l'algorithme.
* Vous pouvez choisir par exemple de tester un modèle linéaire, un modèle à base d'arbres et un modèle de type SVM
* Déterminer le modèle le plus performant parmi ceux testés.

In [ ]:
# CODE COMPARAISON DES MODELES

### Optimisation et interprétation du modèle

A réaliser :
* Reprennez le meilleur algorithme que vous avez sécurisé via l'étape précédente, et réalisez une GridSearch de petite taille sur au moins 3 hyperparamètres.
* Si le meilleur modèle fait partie de la famille des modèles à arbres (RandomForest, GradientBoosting) alors utilisez la fonctionnalité feature importance pour identifier les features les plus impactantes sur la performance du modèle. Sinon, utilisez la méthode Permutation Importance de sklearn. 

In [ ]:
# CODE OPTIMISATION ET INTERPRETATION DU MODELE